In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
%matplotlib inline

from collections import defaultdict
from nltk.stem.snowball import EnglishStemmer  # Assuming we're working with English
from urllib.request import urlopen
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\chide\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chide\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chide\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chide\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [70]:
class Crawler:
    
    
    def manchesterUni_crawler(self, url, data):
    
        html_ = urlopen(url)
        soup = BeautifulSoup(html_, 'lxml')
        links = []
        for a_tag in soup.find_all('a'):
            i = a_tag.get('href')
            li = a_tag.find_parent('li')
            if type(i) == str and i.find('https://www.research.manchester.ac.uk/portal/') != -1:
                links.append(i)
        for n in links:
            html = urlopen(n)
            soup2 = BeautifulSoup(html, 'lxml')
            for re_link in soup2.find_all('a'):
                atag = re_link.get('href')
                if type(atag) == str and atag.find('/projects.html?period=running') != -1:
                    html2 = urlopen(atag)
                    soup3 = BeautifulSoup(html2, 'lxml')
                    divv = soup3.find_all('div', class_ = 'profileinformation')
                    for p in divv:
                        name = soup2.find(class_ = 'person')
                        names = name.text
                        if names and p.text:
                            data.append([names, n , p.text])
    
    def newcastleUni_crawler(self, url, data):  
        html_ = urlopen(url)
        soup = BeautifulSoup(html_, 'lxml')
        links = []
        for a_tag in soup.find_all('a'):
            i = a_tag.get('href')
            li = a_tag.find_parent('li')
            if type(i) == str and i.find('profile/') != -1:
                links.append('https://www.ncl.ac.uk/'+i)
        for n in links:
                html = urlopen(n)
                soup2 = BeautifulSoup(html, 'lxml')
                main = soup2.find(id = 'content')
                article = main.find('article')
                h1 = article.find_all('h1', class_="")
                parent_div = article.find('div', class_='contentSeparator tab')
                for h2 in article.find_all('h2'):
                    if h2.text == 'Research' and h1[0] != '':
                        div_res = h2.find_parent('div')
                        data.append([h1[0].text, n, div_res.text])
    
    def birminghamUni_crawler(self, url, data):
        html_ = urlopen(url)
        soup = BeautifulSoup(html_, 'lxml')
        links = []
        for a_tag in soup.find_all('a'):
            i = a_tag.get('href')
            if i is not None and type(i) == str and (i.find('/schools/gees/people/profile.aspx') != -1 or i.find('staff/profiles/eese/') != -1):
                links.append('https://www.birmingham.ac.uk'+i)
        for n in links:
            yourstring = n.encode('ascii', 'ignore').decode('ascii')
            html = urlopen(yourstring)
            soup2 = BeautifulSoup(html, 'lxml')
            main = soup2.find(id='maincontent')
            article = main.find('article')
            name = article.find('h1', class_='')
            staff = article.find(id='staffdetails')
            if staff is not None:
                for section in staff.find_all('section'):
                    if section is not None:  
                        h2 = section.find('h2')
                        if h2 is not None and h2.text.find('Research') != -1:
                            res = section.text

            for span in name.find_all('span'):
                if span.text in name.text:
                    names = name.text.replace(span.text, '')
                    data.append([names, n, res])
    
    def birmingham2(self, data):
    
        html_ = urlopen('https://www.cs.bham.ac.uk/people/')
        soup = BeautifulSoup(html_, 'lxml')
        links = []
        for a_tag in soup.find_all('a'):
            i = a_tag.get('href')
            if i is not None and type(i) == str and i.find('staff/profiles/computer-science/') != -1:
                 links.append(i)
        for m in links:
            html__ = urlopen(m)
            soup3 = BeautifulSoup(html__, 'lxml')
            main1 = soup3.find(id='maincontent')
            article1 = main1.find('article')
            name = article1.find('h1', class_='')
            staff1 = article1.find(id='staffdetails')
            if staff1 is not None:
                for sections in staff1.find_all('section'):
                    h2 = sections.find('h2')
                    if h2 is not None and h2.text.find('Research') != -1:
                        rese = sections.text
                        for span in name.find_all('span'):
                            if span.text in name.text:
                                prof_name = name.text.replace(span.text, '')
                                data.append([prof_name, m, rese])


In [71]:
crawler = Crawler()

urls = ['https://www.ees.manchester.ac.uk/about/people/academic-and-research-staff/',
        'https://www.mace.manchester.ac.uk/about/people/academic-and-research-staff/', 
        'https://www.cs.manchester.ac.uk/about/people/academic-and-research-staff/'
       ]  # urls to crawl from Manchester Uni

urls1 = ['https://www.ncl.ac.uk/engineering/research/civil/people/#researchstaff',
        'https://www.ncl.ac.uk/computing/people/research/', 
        'https://www.ncl.ac.uk/nes/staff/research/'
       ]  # urls to crawl from Newcastle Uni

urls2 = ['https://www.birmingham.ac.uk/schools/engineering/about/academic.aspx',
        'https://www.birmingham.ac.uk/schools/gees/people/index.aspx', 
       ]  # urls to crawl from Birmingham Uni

dt1 = [] # es
dt2 = [] # ce
dt3 = [] # cs

dt4 = [] #ce
dt5 = [] #cs
dt6 = [] #es

dt7 = [] #ce
dt8 = [] #es
dt9 = [] #cs

crawler.manchesterUni_crawler(urls[0], dt1)
crawler.manchesterUni_crawler(urls[1], dt2)
crawler.manchesterUni_crawler(urls[2], dt3)

crawler.newcastleUni_crawler(urls1[0], dt4)
crawler.newcastleUni_crawler(urls1[1], dt5)
crawler.newcastleUni_crawler(urls1[2], dt6)

crawler.birminghamUni_crawler(urls2[0], dt7)
crawler.birminghamUni_crawler(urls2[1], dt8)
crawler.birmingham2(dt9)

df1 = pd.DataFrame(dt1, columns=['name', 'link', 'research'])
df2 = pd.DataFrame(dt2, columns=['name', 'link', 'research'])
df3 = pd.DataFrame(dt3, columns=['name', 'link', 'research'])

df4 = pd.DataFrame(dt4, columns=['name', 'link', 'research'])
df5 = pd.DataFrame(dt5, columns=['name', 'link', 'research'])
df6 = pd.DataFrame(dt6, columns=['name', 'link', 'research'])

df7 = pd.DataFrame(dt7, columns=['name', 'link', 'research'])
df8 = pd.DataFrame(dt8, columns=['name', 'link', 'research'])
df9 = pd.DataFrame(dt9, columns=['name', 'link', 'research'])

In [75]:
cs_dept = pd.concat([df3, df5, df9], ignore_index=True)
ce_dept = pd.concat([df2, df4, df7], ignore_index=True)
es_dept = pd.concat([df1, df6, df8], ignore_index=True)

In [76]:
def addField(field_data, field = ''):
    data_dept = []
    
    for x in range(len(field_data['name'])):
        x = field
        data_dept.append(x)
    field_data['field'] = data_dept
    return field_data

In [77]:
addField(cs_dept, 'cs')
addField(ce_dept, 'ce')
addField(es_dept, 'es')

,name,link,research,field
0,Professor Grant Allen,https://www.research.manchester.ac.uk/portal/G...,\nResearch interestsGrant is currently active ...,es
1,Professor Grant Allen,https://www.research.manchester.ac.uk/portal/G...,\nOther researchDoctoral Thesis\n\nGrant's PhD...,es
2,Prof Richard Bardgett,https://www.research.manchester.ac.uk/portal/r...,\nResearch interestsMy research is broadly con...,es
3,Dr Stephen Boult,https://www.research.manchester.ac.uk/portal/s...,\nResearch interestsHydrology and hydrochemist...,es
4,"Prof Terence Brown BSc, PhD, FSA",https://www.research.manchester.ac.uk/portal/e...,\nResearch interestsBiomolecular Archaeology\n...,es
...,...,...,...,...
184,Dr Martin Widmann,https://www.birmingham.ac.uk/schools/gees/peop...,\n\n\r\n\t\t\t\t\t\t\tResearch\n\n\nCurrent re...,es
185,Dr Jingsha Xu,https://www.birmingham.ac.uk/schools/gees/peop...,\n\n\r\n\t\t\t\t\t\t\tResearch\n\n\nCurrent re...,es
186,Dr Peng Zhang,https://www.birmingham.ac.uk/schools/gees/peop...,\n\n\r\n\t\t\t\t\t\t\tResearch\n\n\nEnvironmen...,es
187,Dr Zhiling Zhiling Guo,https://www.birmingham.ac.uk/schools/gees/peop...,\n\n\r\n\t\t\t\t\t\t\tResearch\n\n\nResearch I...,es


In [79]:
bigData2 = pd.concat([cs_dept, ce_dept, es_dept], ignore_index=True)
print(len(ce_dept))
print(len(cs_dept))
print(len(es_dept))
print(len(bigData2))

297
64
189
550


In [26]:
bigData2 = bigData2.assign(id=(bigData2['name'].astype('category').cat.codes))
bigData2.to_csv('crawled_uni.csv')

In [2]:
big_df = pd.read_csv('crawled_uni.csv')
big_df.head(10)

,Unnamed: 0,name,link,research,field,id
0,0,Prof Sophia Ananiadou,https://www.research.manchester.ac.uk/portal/s...,\nResearch interestsProf. Ananiadou's main con...,cs,413
1,1,Dr Richard Banach,https://www.research.manchester.ac.uk/portal/r...,\nResearch interestsFor accurate and up to dat...,cs,267
2,2,Prof Gavin Brown,https://www.research.manchester.ac.uk/portal/g...,\nResearch interestsMachine Learning.\n,cs,375
3,3,Dr Ke Chen,https://www.research.manchester.ac.uk/portal/k...,\nResearch interestsMy general areas of intere...,cs,170
4,4,Prof Timothy Cootes,https://www.research.manchester.ac.uk/portal/t...,\nResearch interestsMedical Image Analysis usi...,cs,419
5,5,Prof Timothy Cootes,https://www.research.manchester.ac.uk/portal/t...,\nMethodological knowledge\r\n\tComputer Visio...,cs,419
6,6,Dr Suzanne Embury,https://www.research.manchester.ac.uk/portal/s...,\nOther research\nProjects\n\n\nInstitute of C...,cs,314
7,7,Prof Steve Furber CBE FRS FREng DFBCS FIET CIT...,https://www.research.manchester.ac.uk/portal/s...,\nResearch interestsAPT Projects\n,cs,414
8,8,Prof Carole Goble,https://www.research.manchester.ac.uk/portal/c...,\nResearch interestsCarole Goble’s research is...,cs,365
9,9,Prof Anthony Goodacre,https://www.research.manchester.ac.uk/portal/j...,\nResearch interestsHaving defined and introdu...,cs,359


In [3]:
entry = big_df.loc[190,:].copy()
entry

Unnamed: 0                                                  190
name                          Dr Maria Sharmina PhD, AMEI, FHEA
link          https://www.research.manchester.ac.uk/portal/m...
research      \nResearch interestsEnergy systemsFood-energy-...
field                                                        ce
id                                                          200
Name: 190, dtype: object

In [4]:
class Build_Indexer:
    
    def __init__(self, posTag, stopwords, lemmatize):
        self.posTag = posTag
        self.stopwords = stopwords
        self.lem = lemmatize
    
    
    def process_word(self, word):
        word = word.lower() #to lowercase
        word = word.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
        word = self.lemma_stop(word)
        return word
        
    def get_word_pos(self, word):
        tag = self.posTag([word])[0][1][0].upper() # convert word uppercase
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV} #identify word meaning

        return tag_dict.get(tag, wordnet.NOUN)
        
    def lemma_stop(self, doc):
        stop = self.stopwords.words('english') # get english stopwords from lib 

        tokens = nltk.word_tokenize(doc) # tokenize the word
        string = ""
        for n in tokens: 
            if n not in stop: #check if word is a stopword
                string += self.lem.lemmatize(n, self.get_word_pos(n)) + " " #lemmatize word
        return string 
    
    def transform_data(self, df):
        df = df
        df['name'] = df['name'].apply(self.process_word) # process each name in document
        df['research'] = df['research'].apply(self.process_word) #process each research entry
        df['text'] = df['name'] + " " + df['research'] 
        drop_cols = ['name', 'link', 'research']
        df = df.drop(drop_cols, axis=1)
        return df
    
    def index_entry(self, entry, index):
        words = entry.text.split() #split the words in an entry
        Id = entry['id']
        
        for word in words:
            if word in index.keys(): # check if word is in keys
                  if Id not in index[word]:
                        index[word].append(Id) # add word to specific id
            else:
                index[word] = [Id]
        return index
    
    def index_data(self, df, index): # get index of the whole data
        for i in range(len(df)):
            entry = df.loc[i,:]
            index = self.index_entry(entry = entry, index = index) #index each entry
        return index
    
    def index_build(self, df, index): #buid the final inverted index of the data
        add = self.transform_data(df)
        index = self.index_data(df = add, index = index)
        return index
    



In [5]:
lem = WordNetLemmatizer()
indexer = Build_Indexer(pos_tag, stopwords, lem)

indexer.process_word(entry['name'])

'dr maria sharmina phd amei fhea '

In [7]:
%time indexer.process_word('JUD, HGHK,  UGJHL')

Wall time: 11 ms


'jud hghk ugjhl '

In [8]:
big_processed = big_df.copy()

In [9]:
big_processed['text'] = big_processed['name'] + " " + big_processed['research']
drop_cols = ['name', 'link', 'research', 'field']
big_processed = big_processed.drop(drop_cols, axis=1)

In [10]:
entry = big_processed.loc[3,:].copy()

ind = indexer.index_entry(entry=entry, index= {})
print(ind)

{'Dr': [170], 'Ke': [170], 'Chen': [170], 'Research': [170], 'interestsMy': [170], 'general': [170], 'areas': [170], 'of': [170], 'interest': [170], 'are': [170], 'artificial': [170], 'intelligence': [170], 'and': [170], 'computational': [170], 'cognitive': [170], 'science.': [170], 'In': [170], 'particular,': [170], 'I': [170], 'have': [170], 'been': [170], 'working': [170], 'in': [170], 'machine': [170], 'learning,': [170], 'pattern': [170], 'recognition,': [170], 'perception,': [170], 'systems': [170], 'their': [170], 'applications': [170], 'intelligent': [170], 'system': [170], 'development': [170], 'for': [170], 'over': [170], 'three': [170], 'decades.': [170], 'With': [170], 'the': [170], 'aforementioned': [170], 'fundamental': [170], 'researches,': [170], 'am': [170], 'keen': [170], 'on': [170], 'developing': [170], 'effective': [170], 'efficient': [170], 'techniques': [170], 'real': [170], 'world': [170], 'applications.': [170], 'For': [170], 'my': [170], 'current': [170], 'res

In [500]:
ind_all = indexer.index_data(big_processed, index = {})
len(ind_all)

23540

In [62]:
idx = indexer.index_build(df= big_df, index = {})
#print(idx)

In [ ]:
query = input('Enter search term: ')

In [118]:
import gensim

word2vec = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/chide/OneDrive/Desktop/Info Retrieval/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [675]:
class Query_Index:
    
    def __init__(self, wordVec, indexer, bigD, index):
        self.wordVec = wordVec
        self.indexer = indexer
        self.bigD = bigD
        self.index = index
  
    def process_query(self, query):
        normalize = indexer.process_word(query)
        return normalize.split()
    
    def get_vec_av(self, wordVec, doc):
        wordVec = self.wordVec
        doc = [word for word in doc if word in wordVec.vocab]
        if len(doc) == 0:
            return np.zeros(300)
        else:
            return np.mean(wordVec[doc], axis=0) 
        
    def prepare_for_ranking(self, df):
        corpus = df[['id', 'research']].copy()
        doc_vecs = {}
        for i in range(len(corpus)):
            row = corpus.loc[i,:]
            text = row.research.split()
        doc_vecs[row.id]= self.get_vec_av(self.wordVec, text)
        doc_vecs = pd.DataFrame.from_dict(data=doc_vecs, orient="index")
        doc_vecs['id'] = doc_vecs.index
        return doc_vecs
    
    
    def lists_intersection(self, lists):
        intersect = list(set.intersection(*map(set, lists)))
        intersect.sort()
        return intersect
    
    def search_en(self, query, index):
        index = self.index
        query_split = self.process_query(query)
        retrieved = []
        for word in query_split:
            if word in index.keys():
                retrieved.append(index[word])
            if len(retrieved)>0:
                result = self.lists_intersection(retrieved)
            else:
                result = [0]
        return result
    
    def connect_id(self, retrieved_ids, df):
        
        return df[df['id'].isin(retrieved_ids)].reset_index(drop=True)
    
    def search_query(self, query):
        result_id = self.search_en(query, self.bigD)
        results = self.connect_id(result_id, self.bigD)
        results = self.rank_results(query, results)
        self.print_results(results)
            
    def cos_similarity(self, a, b):
        dot = np.dot(a, b)
        norma = np.linalg.norm(a)
        normb = np.linalg.norm(b)
        cos = dot / (norma * normb)
        return(cos)
    
    def rank_results(self, query, results):
        query_norm = self.process_query(query)
        query_vec = self.get_vec_av(self.wordVec, query_norm)
        res_vecs = self.connect_id(results['id'], self.prepare_for_ranking(self.bigD))
        cos_array = []
        for i in range(len(res_vecs)):
            doc_vec = res_vecs.loc[i,:].drop(['id'])
            cos_array.append(self.cos_similarity(doc_vec, query_vec))
            results['rank'] = cos_array
            results = results.sort_values('rank', axis=0, ascending=True)
        return results
    
    def print_results(self, result_df):
        for i in range(len(result_df)):
            res = result_df.loc[i, :]
            print(res['name'])
            print(res['research'])
            print(res['field'])
            if i == len(result_df):
                print(res.link)
            else:
                print("{}\n" .format(res.link))

In [603]:
big_D = big_df.drop(['text'], axis=1).copy()

idx = indexer.index_build(df= big_df, index = {})

In [676]:
query_in = Query_Index(word2vec, indexer, big_D, idx)

words = entry.text.split()

%time test_vec = query_in.get_vec_av(word2vec, words)

Wall time: 998 µs


In [637]:
doc_vecs = query_in.prepare_for_ranking(df=big_D)

In [649]:
test = "Computer"
print("User query: {}." .format(test))
test_norm = indexer.process_word(test)
print("Normalized query: {}." .format(test_norm))

User query: Computer.
Normalized query: computer .


In [620]:
results = query_in.search_en("prof", idx)
print(results)

[2, 14, 106, 112, 117, 143, 186, 210, 238, 239, 318, 336, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 431, 443, 448, 455, 459, 461]


In [ ]:
query = input("Search for: ") # search for query
query_in.search_query(query) # display according to rank 


# TEXT CLASSIFICATION

In [479]:
from sklearn.model_selection import train_test_split  
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


tfidfconv = TfidfTransformer()
big_X = big_df['research']

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  

X = vectorizer.fit_transform(big_X).toarray() 
y = big_df['field'] 

In [475]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [511]:
clf = RandomForestClassifier(n_estimators=1000, random_state=0)  
clf.fit(X_train, y_train) 

In [477]:
y_pred = clf.predict(X_test)

In [488]:
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[48  2  8]
 [ 3  4  1]
 [ 6  1 37]]
              precision    recall  f1-score   support

          ce       0.84      0.83      0.83        58
          cs       0.57      0.50      0.53         8
          es       0.80      0.84      0.82        44

    accuracy                           0.81       110
   macro avg       0.74      0.72      0.73       110
weighted avg       0.81      0.81      0.81       110

0.8090909090909091
